In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
from llama_index.llms.ollama import Ollama


c:\Users\localadmin\Desktop\myprojects\linkedinsucks\linkedinbot_venv\Lib\site-packages\pydantic\_internal\_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
documents = SimpleDirectoryReader("mydata").load_data()

In [62]:
system_prompt =  """
You are a job application assistant. You have the data about a person's resume, profile, work profile, and other details. Use those details to accurately answer the questions about the candidate's profile with the data provided and answer in the format requested in the question. Answer concisely. The answer should be less than 5 words. You will be rewarded for shorter but precise answers. **If the answer has numbers, represent them using numerals (e.g., "2" instead of "two"). This is important.**

"""

In [4]:
SimpleInputPrompt("<|USER|>(query_str)<|ASSISTANT|>")

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=[], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='<|USER|>(query_str)<|ASSISTANT|>')

In [5]:
import torch

In [33]:
llm = HuggingFaceLLM(
context_window=4096,
max_new_tokens= 256,
generate_kwargs={ "temperature" : 0.0, "do_sample" : False},
system_prompt= system_prompt,
tokenizer_name="MuntasirAhmed/TinyLlama-1.1B-rag-finetuned-v1.0",
model_name="MuntasirAhmed/TinyLlama-1.1B-rag-finetuned-v1.0",
device_map="auto",
model_kwargs={"torch_dtype":torch.float16 }
#, "load_in_8bit" : True
)

ValueError: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.

In [7]:
from sentence_transformers import SentenceTransformer

In [8]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.legacy.embeddings import LangchainEmbedding

In [63]:
llm2 = Ollama(
model = "phi3:mini",
temperature = 0.01,
request_timeout = 400,
system_prompt = system_prompt,
context_window = 3900

)

In [43]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

c:\Users\localadmin\Desktop\myprojects\linkedinsucks\linkedinbot_venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [64]:
# use service context for bundling everything togetherr, documents, embeddings, llamaindex etc

service_context = ServiceContext.from_defaults(
chunk_size= 1024,
llm = llm2,
embed_model= embed_model


)

C:\Users\localadmin\AppData\Local\Temp\ipykernel_23796\3444695786.py:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [15]:
torch.cuda.is_available()

False

In [65]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [46]:
device = torch.device("cuda")
print(device)

cuda


In [66]:
query_engine = index.as_query_engine()


In [26]:
HuggingFaceLLM?

Init signature:
HuggingFaceLLM(
    context_window: int = 3900,
    max_new_tokens: int = 256,
    query_wrapper_prompt: Union[str, llama_index.core.prompts.base.PromptTemplate] = '{query_str}',
    tokenizer_name: str = 'StabilityAI/stablelm-tuned-alpha-3b',
    model_name: str = 'StabilityAI/stablelm-tuned-alpha-3b',
    model: Optional[Any] = None,
    tokenizer: Optional[Any] = None,
    device_map: Optional[str] = 'auto',
    stopping_ids: Optional[List[int]] = None,
    tokenizer_kwargs: Optional[dict] = None,
    tokenizer_outputs_to_remove: Optional[list] = None,
    model_kwargs: Optional[dict] = None,
    generate_kwargs: Optional[dict] = None,
    is_chat_model: Optional[bool] = False,
    callback_manager: Optional[llama_index.core.callbacks.base.CallbackManager] = None,
    system_prompt: str = '',
    messages_to_prompt: Optional[Callable[[Sequence[llama_index.core.base.llms.types.ChatMessage]], str]] = None,
    completion_to_prompt: Optional[Callable[[str], str]] = None

In [67]:
response = query_engine.query("how many years of experience do you have with SQL")
response

Response(response='Two years', source_nodes=[NodeWithScore(node=TextNode(id_='23ff23b9-570c-4972-a9bb-e3cbccd8aa1b', embedding=None, metadata={'page_label': '1', 'file_name': 'resume_july_2024_master.pdf', 'file_path': 'c:\\Users\\localadmin\\Desktop\\myprojects\\linkedinsucks\\linkedinsucks_2\\RAG_models\\mydata\\resume_july_2024_master.pdf', 'file_type': 'application/pdf', 'file_size': 191973, 'creation_date': '2024-08-08', 'last_modified_date': '2024-08-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='534d1c80-ee49-4400-b4ca-bcd506683e48', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'resume_july_2024_master.pdf', 'file_path': 'c:\\Users\\localadmin\\Desktop\\mypro

In [23]:
import torch
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))


AssertionError: Torch not compiled with CUDA enabled